In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lab3').getOrCreate()

In [6]:
df=spark.read.option("recursiveFileLookup", "true").csv("./donation")
df.show(2)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
|53719|60579|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
only showing top 2 rows



In [7]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import col, lower, regexp_replace

In [8]:
df2=spark.read.option("header", "true").option("nullValue", "?").\
option("inferSchema", "true").csv("donation/block_1.csv")

def preprocess_data(df):
    df = df.withColumn("clean_text", lower(col("cmp_bd")))
    df = df.withColumn("clean_text", regexp_replace(col("clean_text"), "[^a-zA-Z0-9\\s]", ""))
    tokenizer = Tokenizer(inputCol="clean_text", outputCol="tokens")
    df = tokenizer.transform(df)
    remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
    df = remover.transform(df)
    return df
preprocessed_df2 = preprocess_data(df2)
preprocessed_df2.show(truncate=False)


+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+----------+------+---------------+
|id_1 |id_2 |cmp_fname_c1     |cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|clean_text|tokens|filtered_tokens|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+----------+------+---------------+
|37291|53113|0.833333333333333|null        |1.0         |null        |1      |1     |1     |1     |0      |true    |1         |[1]   |[1]            |
|39086|47614|1.0              |null        |1.0         |null        |1      |1     |1     |1     |1      |true    |1         |[1]   |[1]            |
|70031|70237|1.0              |null        |1.0         |null        |1      |1     |1     |1     |1      |true    |1         |[1]   |[1]            |
|84795|97439|1.0              |null        |1.0         |null        |1      |1     |1     |1 

In [9]:
df2.groupBy("is_match").count().orderBy(col("count").desc()).show()
df2.createOrReplaceTempView("linkage")
spark.sql("""
    SELECT is_match, COUNT(*) cnt
    FROM linkage
    GROUP BY is_match
    ORDER BY cnt DESC
""").show()

summary=df2.describe()
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()
matches=df2.where("is_match = true")
match_summary=matches.describe()
misses=df2.filter(col("is_match")==False)
miss_summary=misses.describe()

+--------+------+
|is_match| count|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+

+--------+------+
|is_match|   cnt|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|            574811|             10325|
|   mean|0.7127592938253411|0.8977586763518969|
| stddev|0.3889286452463531|0.2742577520430532|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [10]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import expr
from pyspark.sql.types import DoubleType

In [11]:
!pip3 install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable


In [22]:
# import decs
def pivot_summary(desc):
    desc_p=desc.toPandas
    desc_p=desc_p.set_index('summary').transpose().reset_index()
    desc_p=desc_p.rename(columns={'index':'field'})
    desc_p=desc_p.rename_axis(None, axis=1)
    descT=spark.createDataFrame(desc_p)
    for c in descT.columns:
        if c=='field':
            continue
        else:
            descT=descT.withColumn(c, descT[c].cast(DoubleType()))
    return descT

match_summaryT=pivot_summary(match_summary)
miss_summaryT=pivot_summary(miss_summary)
good_features=["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]
# sum_expression=" + ".join(good_features)

scored=df2.fillna(0, subset=good_features).\
                withColumn('score', expr(sum_expression)).\
                select('score', 'is_match')

scored.show()

def crossTabs(scored: DataFrame, t: float) -> DataFrame:
    return scored.selectExpr(f"score >= {t} as above", "is_match").\
        groupBy("above").pivot("is_match", ("true", "false")).\
        count()

confused=crossTabs(scored, 4.0)
confused.show()
confused2=crossTabs(scored, 2.0)
confused2.show()

tp=confused.filter("above = true").select("true").collect()[0].true
fp=confused.filter("above = true").select("false").collect()[0].false
fn=confused.filter("above = false").select("true").fillna(0).collect()[0].true
tn=confused.filter("above = false").select("false").collect()[0].false

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)

# precision, recall, f1

AttributeError: 'function' object has no attribute 'set_index'